In [6]:
from PIL import Image
import numpy as np
import glob
import os
from sklearn.metrics import jaccard_similarity_score

In [7]:
def get1DArray(img_path):
    
    return np.array(Image.open(img_path).convert("RGBA")).flatten()

def getAllImagesOfPathIn1D(dir_path):
    #image_list = map(Image.open, glob(dir_path+'*.png'))
    image_list = list()
    for root, directories, filenames in os.walk(dir_path+"/"):
        for filename in filenames: 
            image_list.append((filename,get1DArray(os.path.join(root,filename))))
    return image_list

avg_dir = os.path.join('data','100TestImages','average')
avg_files = getAllImagesOfPathIn1D(avg_dir)
lime_dir = os.path.join('data','lime_erg','cell_images','27-03-2019 1254')
#print(lime_dir)
WS_true = list()
SL_true = list()
FW_true = list()
QS_true = list()
QSopt_true = list()

WS_false = list()
SL_false = list()
FW_false = list()
QS_false = list()

pred_true = set()
pred_false = set()
greyImage = get1DArray('data/100TestImages/grey6464.png')
for avg_file in avg_files:    
    avg_path_name = avg_file[0].replace(".png","")    
    filecontent = open(lime_dir+"/"+avg_path_name+"/prediction", 'r').read()
    #print(filecontent)
    pred = filecontent[5:filecontent.find("real")-1]
    real = filecontent[filecontent.find("real")+len("real")+1:]
    for file in glob.glob(lime_dir+"/"+avg_path_name+"/"+'/*png'):
        if pred == real:# or 1==1: 
            pred_true.add(avg_path_name)
            file_name = file[file.rfind("/")+1:]
            file_name = file_name[file_name.rfind("\\")+1:]
            if not file_name.startswith("org"):            
                if file_name.startswith("felzenszwalb"):
                    FW_true.append((file_name,get1DArray(file)))
                elif file_name.startswith("quickshiftoptimized"):                    
                    QSopt_true.append((file_name,get1DArray(file)))
                elif file_name.startswith("quickshift"):                    
                    QS_true.append((file_name,get1DArray(file)))
                elif file_name.startswith("slic"):
                    SL_true.append((file_name,get1DArray(file)))
                elif file_name.startswith("watershed"):
                    WS_true.append((file_name,get1DArray(file)))          
                   
                    #raise Exception('wrong filename')
        else:
            pred_false.add(avg_path_name)
            file_name = file[file.rfind("/")+1:]
            file_name = file[file.rfind("\\")+1:]
            if not file_name.startswith("org"):            
                if file_name.startswith("felzenszwalb"):
                    FW_false.append((file_name,get1DArray(file)))
                elif file_name.startswith("quickshift"):                    
                    QS_false.append((file_name,get1DArray(file)))
                elif file_name.startswith("slic"):
                    SL_false.append((file_name,get1DArray(file)))
                elif file_name.startswith("watershed"):
                    WS_false.append((file_name,get1DArray(file)))
                #else:
                #    raise Exception('wrong filename')

In [8]:
FW_all = []
QS_all = []
QSopt_all = []
SL_all = []
WS_all = []
#FW_sumt = 0
#QS_sumt = 0
#SL_sumt = 0
#WS_sumt = 0
#print(FW_true)
for i in range(len(FW_true)):
    FW_all.append(jaccard_similarity_score(avg_files[i][1],FW_true[i][1]))    
    QS_all.append(jaccard_similarity_score(avg_files[i][1],QS_true[i][1]))
    QSopt_all.append(jaccard_similarity_score(avg_files[i][1],QSopt_true[i][1]))
    SL_all.append(jaccard_similarity_score(avg_files[i][1],SL_true[i][1]))
    WS_all.append(jaccard_similarity_score(avg_files[i][1],WS_true[i][1]))
    
#for i in range(len(FW_false)):
#    FW_sumf += jaccard_similarity_score(org_files[i][1],FW_false[i][1])    
#    QS_sumf += jaccard_similarity_score(org_files[i][1],QS_false[i][1])
#    SL_sumf += jaccard_similarity_score(org_files[i][1],SL_false[i][1])
#    WS_sumf += jaccard_similarity_score(org_files[i][1],WS_false[i][1])

In [10]:
from statistics import mean,variance,stdev,median




print("Jaccard-Koeffizent mean (prediction == true({}))".format(len(FW_true)))
print("felzenszwalb : {0:.8f}".format(mean(FW_all)))
print("quickshift: {0:.8f}".format(mean(QS_all)))
print("quickshift optimized: {0:.8f}".format(mean(QSopt_all)))
print("slic: {0:.8f}".format(mean(SL_all)))
print("compact watershed: {0:.8f}".format(mean(WS_all)))
print("-----")
print("Jaccard-Koeffizent median (prediction == true({}))".format(len(FW_true)))
print("felzenszwalb : {0:.8f}".format(median(FW_all)))
print("quickshift: {0:.8f}".format(median(QS_all)))
print("quickshift optimized: {0:.8f}".format(median(QSopt_all)))
print("slic: {0:.8f}".format(median(SL_all)))
print("compact watershed: {0:.8f}".format(median(WS_all)))
print("-----")
print("Jaccard-Koeffizent variance (prediction == true({}))".format(len(FW_true)))
print("felzenszwalb : {0:.8f}".format(variance(FW_all)))
print("quickshift: {0:.8f}".format(variance(QS_all)))
print("quickshift optimized: {0:.8f}".format(variance(QSopt_all)))
print("slic: {0:.8f}".format(variance(SL_all)))
print("compact watershed: {0:.8f}".format(variance(WS_all)))
print("-----")
print("Jaccard-Koeffizent standard Abweichung (prediction == true({}))".format(len(FW_true)))
print("felzenszwalb : {0:.8f}".format(stdev(FW_all)))
print("quickshift: {0:.8f}".format(stdev(QS_all)))
print("quickshift optimized: {0:.8f}".format(stdev(QSopt_all)))
print("slic: {0:.8f}".format(stdev(SL_all)))
print("compact watershed: {0:.8f}".format(stdev(WS_all)))
print("-----")
print("Jaccard-Koeffizent: Coefficient of variation(prediction == true({}))".format(len(FW_true)))
print("felzenszwalb : {0:.8f}".format(stdev(FW_all)/mean(FW_all)))
print("quickshift: {0:.8f}".format(stdev(QS_all)/mean(QS_all)))
print("quickshift optimized: {0:.8f}".format(stdev(QSopt_all)/mean(QSopt_all)))
print("slic: {0:.8f}".format(stdev(SL_all)/mean(SL_all)))
print("compact watershed: {0:.8f}".format(stdev(WS_all)/mean(WS_all)))
print("-----")

Jaccard-Koeffizent mean (prediction == true(85))
felzenszwalb : 0.85603243
quickshift: 0.52272303
quickshift optimized: 0.88820585
slic: 0.96437629
watershed: 0.97850773
-----
Jaccard-Koeffizent median (prediction == true(85))
felzenszwalb : 0.96502686
quickshift: 0.56054688
quickshift optimized: 0.90325928
slic: 0.96582031
watershed: 0.97869873
-----
Jaccard-Koeffizent variance (prediction == true(85))
felzenszwalb : 0.03330687
quickshift: 0.04613085
quickshift optimized: 0.00307818
slic: 0.00014387
watershed: 0.00003847
-----
Jaccard-Koeffizent standard Abweichung (prediction == true(85))
felzenszwalb : 0.18250170
quickshift: 0.21478094
quickshift optimized: 0.05548137
slic: 0.01199452
watershed: 0.00620228
-----
Jaccard-Koeffizent: Coefficient of variation(prediction == true(85))
felzenszwalb : 0.21319484
quickshift: 0.41088860
quickshift optimized: 0.06246453
slic: 0.01243759
watershed: 0.00633851
-----
